In [ ]:
#Use this code whenever we've updated the environment.yml document
#%conda env update --file environment.yml  --prune

In [8]:
import scipy as sp
import numpy as np
import os

from dask_image import ndmeasure
from dask import array as da
from dask.distributed import Client, progress

import napari

# Label The Fingers

This notebook takes a Dask Stack of NumPy Arrays representing a hand, and

 1. Sets a threshold that will separate the proximal phalanges from the metacarpals
 2. Labels the bones
 3. Removes small labels
 4. Relabels index = 1, middle = 2, ring = 3, pinky = 4
 5. Saves as a dask stack of numpy arrays.

In [ ]:
client = Client(processes=False, threads_per_worker=6,
                n_workers=1, memory_limit='12GB')
client

### User Input

These are the user inputs needed for this algorithm to work properly.  Specifically:
 - `specific` is the name of the folder where the input data is located.
 - `H0` is the lowest Houndsfield limit such that the fingers are totally disconnected from the metacarpals.
 - `small` is the limit such that all labels of objects with less then this number of voxels are thrown away.

In [9]:
specific = 'S232028'
H0 = 450
small = 20**3

Next, we define the various directories we will need, and we will create them if they do not yet already exist.

In [10]:
hand_dir = '../hands/' + specific + '/'
temp_finger_labels = '../hands/' + specific + '_temp_finger_labels/'
temp_dir = 'temp/'
for d in [temp_finger_labels, temp_dir]:
    os.makedirs(d, exist_ok=True)

In [11]:
def empty_the_folder(directory):
    #This method deletes all files from a temporary folder.
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if os.path.isfile(file_path):
            os.remove(file_path)
    print(f'removed files from {directory}')

In [12]:
empty_the_folder(temp_dir)

removed files from temp/


In [13]:
hand = da.from_npy_stack(hand_dir)
spacing = np.load('../spacing/' + specific + '.npy')

### User Input needed

Use napari below to determine the minimum threshold for the bones such that the metacarpals and the fingers are distinct.  Then change `H0` above to match this number.

In [25]:
viewer = napari.Viewer()
hand_view = viewer.add_image(
    hand,
    scale=spacing,
    contrast_limits = (H0, H0+1),
)

In [14]:
bones_H0 = (hand > H0).rechunk()
bones_H0

dask.array<rechunk-merge, shape=(1383, 1325, 1827), dtype=bool, chunksize=(55, 1325, 1827), chunktype=numpy.ndarray>

In [15]:
da.to_npy_stack(temp_dir, bones_H0)

In [16]:
#Note: this runs very slowly if you do not include the ".compute()".
#Perhaps there is a better dask-ish way to d this?
#labels, num_labels = ndmeasure.label(bones_H0.compute())
%time labels, num_labels = ndmeasure.label(bones_H0)

CPU times: total: 4min 36s
Wall time: 4min 36s


In [17]:
labels

dask.array<getitem, shape=(1383, 1325, 1827), dtype=int32, chunksize=(55, 1325, 1827), chunktype=numpy.ndarray>

In [19]:
#Compute the labels and save them here.  We will overwrite when we remove the small objects.
da.to_npy_stack(temp_finger_labels, labels)

In [20]:
#Reloading it from the disk here will keep Dask from recomputing everytime we need them.
labels = da.from_npy_stack(temp_finger_labels)
labels

dask.array<from-npy-stack, shape=(1383, 1325, 1827), dtype=int32, chunksize=(55, 1325, 1827), chunktype=numpy.ndarray>

In [ ]:
counts = da.bincount(labels.ravel()).compute()
#print(counts)

In [22]:
# Exclude the background label
counts[0]=0
# Determine how many labels to relabel.  All other labels will be removed.
end = sum(counts > small)  #This will help us know when to stop after we sort the labels by size.
# Sort the labels according to their sizes, and stop based on the number determined above
sorted_labels = np.argsort(counts)[:-end-1:-1]
# Create a label map that sends most things to zero and relabels the big labels
label_map = np.zeros(counts.shape, dtype=np.dtype('uint8'))
for new_label, old_label in enumerate(sorted_labels, start=1):
    label_map[old_label] = new_label
#Use the label map to relabel
def relabel_and_remove_small(label_block):
    return label_map[label_block]
labels = labels.map_blocks(relabel_and_remove_small)
labels

dask.array<relabel_and_remove_small, shape=(1383, 1325, 1827), dtype=uint8, chunksize=(55, 1325, 1827), chunktype=numpy.ndarray>

In [23]:
#overwrite the last version of labels with the relabelled and cleaned version
da.to_npy_stack(temp_finger_labels, labels)

In [24]:
labels = da.from_npy_stack(temp_finger_labels)

## Human Intervention Potentiall Needed Here

Look at the labels in Napari, and relabel them so that 
- Index = 1
- Middle = 2
- Ring = 3
- Pinky = 4
- All others = 5

If the hand picture only shows parts of the metacarpals, then it is likely that currently the middle is 1, the ring is 2, the index is 3, the thumb is 4, and the pinky is 5 because of their volumes.  The second-most likely possibility is that the thumb is 5 and the pinky is 4.  Sometimes a large metalic oject shifts these all down by 1.

In [ ]:
viewer.add_labels(
    labels,
    name=f'bones > {H0} labeled',
    opacity=1.0,
    scale=spacing,
)

In [ ]:
#                      0, 1, 2, 3, 4, 5, ...
finger_map = np.array([0, 5, 2, 1, 3, 4] + [5]*(end-5)).astype(np.dtype('int8'))
def relabel_fingers(label_block):
    return finger_map[label_block]
labels = labels.map_blocks(relabel_fingers)

In [ ]:
da.to_npy_stack(temp_finger_labels, labels)

In [ ]:
labels = da.from_npy_stack(temp_finger_labels)

In [ ]:
labels

In [ ]:
viewer.add_labels(
    labels,
    name='fingers',
    opacity=1.0,
    scale=spacing,
)